In [1]:
import pandas as pd

from src.path import IPOPT_PATH
import src.constants as cst
import src.model as mod

In [2]:
profiles = pd.DataFrame(
    {
        "Clément": {cst.TECH: cst.LVB, cst.AUT: cst.LVA, cst.EXP: cst.LVD, cst.REL: cst.LVC, cst.SLRY: 467},
        "Maxence": {cst.TECH: cst.LVB, cst.AUT: cst.LVD, cst.EXP: cst.LVD, cst.REL: cst.LVD, cst.SLRY: 321},
        "Josiane": {cst.TECH: cst.LVD, cst.AUT: cst.LVD, cst.EXP: cst.LVA, cst.REL: cst.LVB, cst.SLRY: 373},
        "Jiminy": {cst.TECH: cst.LVD, cst.AUT: cst.LVB, cst.EXP: cst.LVA, cst.REL: cst.LVA, cst.SLRY: 482},
    }
)

profiles

,Clément,Maxence,Josiane,Jiminy
Technicité,B,B,D,D
Autonomie,A,D,D,B
Exposition,D,D,A,A
Relations,C,D,B,A
Salaire,467,321,373,482


In [3]:
weights = pd.DataFrame(
    {
        cst.TECH: {cst.LVA: 50, cst.LVB: 40, cst.LVC: 30, cst.LVD: 20},
        cst.AUT: {cst.LVA: 10, cst.LVB: 8, cst.LVC: 6, cst.LVD: 4},
        cst.EXP: {cst.LVA: 100, cst.LVB: 80, cst.LVC: 60, cst.LVD: 40},
        cst.REL: {cst.LVA: 5, cst.LVB: 4, cst.LVC: 3, cst.LVD: 2},
    }
)

weights

,Technicité,Autonomie,Exposition,Relations
A,50,10,100,5
B,40,8,80,4
C,30,6,60,3
D,20,4,40,2


### Model

#### Solve Model

In [4]:
weights_sol, obj = mod.solve_model(
    skills=profiles.loc[cst.SKILLS],
    salaries=profiles.loc[cst.SLRY],
    levels=cst.LEVELS,
    ipopt_path=IPOPT_PATH,
)
print(f"Error: {obj}")
weights_sol

Error: 1.783909507990845e-16


,A,B,C,D,Gap
Autonomie,163.762555,126.762555,89.762555,52.762555,37.000000
Exposition,105.483943,104.218636,102.953329,101.688022,1.265307
Relations,174.965874,139.965874,104.965873,69.965873,35.000000
Technicité,107.481510,96.583549,85.685589,74.787629,10.897960


### Check

Can use 'compute-salary' function direclty to series

In [5]:
from_sol = pd.Series(
    {
        per: mod.compute_salary(profile=val[cst.SKILLS], weights=weights_sol)
        for per, val in profiles.items()
    }
)

pd.concat([from_sol, profiles.loc[cst.SLRY]], axis=1, keys=["Solution", "Original"])

,Solution,Original
Clément,467.0,467
Maxence,321.0,321
Josiane,373.0,373
Jiminy,482.0,482


Some specific values can be fixed, however if too many constraints are added, the problem might become infeasible

### Solve with Fixed gaps

In [6]:
weights_sol, obj = mod.solve_model(
    skills=profiles.loc[cst.SKILLS],
    salaries=profiles.loc[cst.SLRY],
    levels=cst.LEVELS,
    ipopt_path=IPOPT_PATH,
    fix_gaps={cst.TECH: 45}
)
print(f"Error: {obj}")
weights_sol

Error: 4.160384385352303e-18


,A,B,C,D,Gap
Autonomie,185.188751,148.188751,111.188751,74.188751,37.0
Exposition,150.407331,126.407331,102.407331,78.407331,24.0
Relations,169.366276,134.366276,99.366276,64.366276,35.0
Technicité,149.037643,104.037643,59.037643,14.037643,45.0


### Solve with fixed levels

In [7]:
weights_sol, obj = mod.solve_model(
    skills=profiles.loc[cst.SKILLS],
    salaries=profiles.loc[cst.SLRY],
    levels=cst.LEVELS,
    ipopt_path=IPOPT_PATH,
    fix_levels={(skl, cst.LVD): 50 for skl in cst.SKILLS}
)
print(f"Error: {obj}")
weights_sol

Error: 5.461642794026597e-19


,A,B,C,D,Gap
Autonomie,161.0,124.000000,87.000000,50.0,37.000000
Exposition,153.0,118.666667,84.333333,50.0,34.333333
Relations,155.0,120.000000,85.000000,50.0,35.000000
Technicité,231.5,171.000000,110.500000,50.0,60.500000
